In [134]:
import os
import sys
sys.path.append(os.path.join('..','data_collect'))
import gc
import numpy as np
import pandas as pd
import JS_NV_crawler as JS
from tqdm import tqdm

# 데이터 로딩

In [227]:
# 네이버 서울 음식점
nv_place = pd.read_excel(os.path.join("..","..", "data","JS_0601_select_place.xlsx"))

In [174]:
# 트립어드바이저 서울 음식점
ta_place = pd.read_excel(os.path.join("..","..", "data","TA_음식점_한글이름_주소_정보.xlsx"))

In [7]:
nv_place.shape

(9867, 18)

In [8]:
ta_place.shape

(5574, 7)

In [67]:
# 이름만으로 매칭되는 것은 900개 정도
nv_place.merge(ta_place, left_on = 'place.name', right_on = 'name', how = 'inner')

In [109]:
# 주소로만 매칭하는 것도 안될 것 같음, 심지어 위경도도 같아버림
nv_place.merge(ta_place, on = 'land.addr', how = 'inner').head(5)[['place.name', 'name', 'land.addr', 'lat_x', 'lng_x', 'lat_y', 'lng_y']]

,place.name,name,land.addr,lat_x,lng_x,lat_y,lng_y
0,안동찜닭&해물떡찜,커피빈 삼성오크우드점,서울특별시 강남구 삼성동 159 코엑스,37.511662,127.059427,37.511662,127.059427
1,안동찜닭&해물떡찜,롯데리아,서울특별시 강남구 삼성동 159 코엑스,37.511662,127.059427,37.511662,127.059427
2,안동찜닭&해물떡찜,테이블스타,서울특별시 강남구 삼성동 159 코엑스,37.511662,127.059427,37.511662,127.059427
3,안동찜닭&해물떡찜,하동관 코엑스몰직영점,서울특별시 강남구 삼성동 159 코엑스,37.511662,127.059427,37.511662,127.059427
4,안동찜닭&해물떡찜,테라로사,서울특별시 강남구 삼성동 159 코엑스,37.511662,127.059427,37.511662,127.059427


# ta 위경도 값 없는 것 가져오기

In [71]:
addrs = ta_place['address']

In [72]:
from tqdm import tqdm

In [73]:
addr_list = []
for addr in tqdm(addrs):
    value = JS.NVgeocoding(addr)
    addr_list.append(value)

100%|██████████| 5574/5574 [08:57<00:00, 10.37it/s] 


# 트립어드바이저 새로운 주소 붙여주기

In [35]:
temp = pd.DataFrame(addr_list, columns =['land.addr', 'road.addr', 'lat', 'lng'])

In [175]:
ta_place = pd.concat([ta_place, temp], axis = 1)

In [205]:
# 트립어드바이저 서울 음식점 위치 네이버로 업데이트
ta_place.to_excel(os.path.join("..","..", "data","TA_음식점_한글이름_주소_정보.xlsx"))

# nv 와 ta 거리 계산하기 9000 * 5000

In [158]:
from scipy.spatial import distance

In [110]:
nv_place['lat'] = nv_place['lat'].astype(float)
nv_place['lng'] = nv_place['lng'].astype(float)
ta_place['lat'] = ta_place['lat'].astype(float)
ta_place['lng'] = ta_place['lng'].astype(float)

In [112]:
i = 0
total = []
for args in tqdm(nv_place[['place.id','lat', 'lng']].values):
    nv_idx = args[0]
    n_lat, n_lng = args[1:]
    dis_all = []
    for kwargs in ta_place[['location_id', 'lat', 'lng']].values:
        ta_idx = kwargs[0]
        t_lat, t_lng = kwargs[1:]
        dis = distance.euclidean((n_lat, n_lng), (t_lat, t_lng))
        value = [nv_idx, ta_idx, dis]
        dis_all.append(value)
        
    if float(min(np.array(dis_all)[:,2])) < 0.0001:
        loc = np.argmin(np.array(dis_all)[:,2])
        value = np.array(dis_all)[loc]
    else:
        value = 'none'
    i += 1
    total.append(value)

100%|██████████| 9867/9867 [16:27<00:00,  9.99it/s]


In [113]:
len(total)

9867

In [114]:
total = pd.DataFrame(total)

In [139]:
total['is_valid'] = total[0].apply(lambda x: 1 if len(str(x).split(',')[0]) > 4 else 0)

In [141]:
total = total.loc[total['is_valid'] == 1]

In [161]:
total['place.id'] = total[0].apply(lambda x : x[0])
total['location_id'] = total[0].apply(lambda x : x[1])

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [163]:
total.drop([0,'is_valid'], axis = 1, inplace = True)

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [164]:
total

,place.id,location_id
14,33879152,9141260.0
34,36166539,3938485.0
62,11605455,2241571.0
81,1266909878,9318901.0
84,1288556612,3911442.0
...,...,...
9839,35162551,9029313.0
9840,36667369,8942305.0
9843,37204578,4034834.0
9844,38404658,9402393.0


In [159]:
def matchName(nv, ta):
    nv_name = nv_place.loc[nv_place['place.id'] == nv, 'place.name'].values[0]
    ta_name = ta_place.loc[ta_place['location_id'] == float(ta), 'name'].values[0]
    return nv_name, ta_name

In [165]:
matchName(*total.values[:,:2][0])

('부산아구', '첨벙아구찜')

In [166]:
total.to_excel(os.path.join("..","..", "data","JS_09_between_distance.xlsx"))

# idx 기준 주소와 이름 가져오기

In [180]:
dt = total.copy()

In [177]:
nv_place_t = nv_place[['place.id', 'place.name', 'land.addr']]

In [178]:
ta_place_t = ta_place[['location_id', 'name', 'land.addr']]

In [181]:
dt = dt.merge(nv_place_t, on = 'place.id', how = 'left')

In [182]:
dt['location_id']

0        9141260.0
1        3938485.0
2        2241571.0
3        9318901.0
4        3911442.0
           ...    
1063     9029313.0
1064     8942305.0
1065     4034834.0
1066     9402393.0
1067    13964808.0
Name: location_id, Length: 1068, dtype: object

In [183]:
dt['location_id'] = dt['location_id'].apply(lambda x : int(float(x)) if x != 'o' else x)

In [184]:
dt = dt.merge(ta_place_t, on = 'location_id', how = 'left')

In [185]:
dt

,place.id,location_id,place.name,land.addr_x,name,land.addr_y
0,33879152,9141260,안동찜닭&해물떡찜,서울특별시 강남구 삼성동 159 코엑스,커피빈 삼성오크우드점,서울특별시 강남구 삼성동 159 코엑스
1,36166539,3938485,에스키모 코엑스몰점,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울,아웃백 코엑스점,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울
2,11605455,2241571,가미분식,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,가미 분식,서울특별시 서대문구 대현동 54-1 무궁화상가아파트
3,1266909878,9318901,우리두리,서울특별시 용산구 동자동 43-205 한국철도공사,파리크라상,서울특별시 용산구 동자동 43-205 한국철도공사
4,1288556612,3911442,마포만두,서울특별시 양천구 목1동 404-177 썬택씨티빌딩,스타벅스 오목교점,서울특별시 양천구 목동 404-177 썬택씨티빌딩
...,...,...,...,...,...,...
1063,35162551,9029313,계절밥상 용산아이파크몰점,서울특별시 용산구 한강로3가 40-999 용산역,계절밥상 - 용산아이파크몰점,서울특별시 용산구 한강로3가 40-999 용산역
1064,36667369,8942305,계절밥상 동대문롯데피트인점,서울특별시 중구 을지로6가 21-31 롯데 FiTin,엔제리너스,서울특별시 중구 을지로6가 21-31 롯데 FiTin
1065,37204578,4034834,계절밥상 잠실점,서울특별시 송파구 신천동 7-18 롯데캐슬골드,무스쿠스 잠실점,서울특별시 송파구 신천동 7-18 롯데캐슬골드
1066,38404658,9402393,계절밥상 서울역사점,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사,맥도날드,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사


In [187]:
dt = dt[['place.id', 'location_id', 'place.name', 'name', 'land.addr_x', 'land.addr_y']]

In [188]:
dt

,place.id,location_id,place.name,name,land.addr_x,land.addr_y
0,33879152,9141260,안동찜닭&해물떡찜,커피빈 삼성오크우드점,서울특별시 강남구 삼성동 159 코엑스,서울특별시 강남구 삼성동 159 코엑스
1,36166539,3938485,에스키모 코엑스몰점,아웃백 코엑스점,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울
2,11605455,2241571,가미분식,가미 분식,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,서울특별시 서대문구 대현동 54-1 무궁화상가아파트
3,1266909878,9318901,우리두리,파리크라상,서울특별시 용산구 동자동 43-205 한국철도공사,서울특별시 용산구 동자동 43-205 한국철도공사
4,1288556612,3911442,마포만두,스타벅스 오목교점,서울특별시 양천구 목1동 404-177 썬택씨티빌딩,서울특별시 양천구 목동 404-177 썬택씨티빌딩
...,...,...,...,...,...,...
1063,35162551,9029313,계절밥상 용산아이파크몰점,계절밥상 - 용산아이파크몰점,서울특별시 용산구 한강로3가 40-999 용산역,서울특별시 용산구 한강로3가 40-999 용산역
1064,36667369,8942305,계절밥상 동대문롯데피트인점,엔제리너스,서울특별시 중구 을지로6가 21-31 롯데 FiTin,서울특별시 중구 을지로6가 21-31 롯데 FiTin
1065,37204578,4034834,계절밥상 잠실점,무스쿠스 잠실점,서울특별시 송파구 신천동 7-18 롯데캐슬골드,서울특별시 송파구 신천동 7-18 롯데캐슬골드
1066,38404658,9402393,계절밥상 서울역사점,맥도날드,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사


# 서로서로 이름이 속하면 같은 매장으로

In [189]:
arg = dt.values[0]

In [191]:
arg[2], arg[3]

('안동찜닭&해물떡찜', '커피빈 삼성오크우드점')

In [192]:
arg[3].replace(' ','')

'커피빈삼성오크우드점'

In [201]:
name_duplicate = []
for arg in dt.values:
    nv_name = arg[2].replace(' ', '')
    ta_name = arg[3].replace(' ', '')
    if nv_name in ta_name:
        value = 1
    elif ta_name in nv_name:
        value = 1
    else:
        value = 0
    name_duplicate.append(value)

In [202]:
dt['name_duplicate'] = name_duplicate

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [203]:
dt

,place.id,location_id,place.name,name,land.addr_x,land.addr_y,name_duplicate
0,33879152,9141260,안동찜닭&해물떡찜,커피빈 삼성오크우드점,서울특별시 강남구 삼성동 159 코엑스,서울특별시 강남구 삼성동 159 코엑스,0
1,36166539,3938485,에스키모 코엑스몰점,아웃백 코엑스점,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울,0
2,11605455,2241571,가미분식,가미 분식,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,1
3,1266909878,9318901,우리두리,파리크라상,서울특별시 용산구 동자동 43-205 한국철도공사,서울특별시 용산구 동자동 43-205 한국철도공사,0
4,1288556612,3911442,마포만두,스타벅스 오목교점,서울특별시 양천구 목1동 404-177 썬택씨티빌딩,서울특별시 양천구 목동 404-177 썬택씨티빌딩,0
...,...,...,...,...,...,...,...
1063,35162551,9029313,계절밥상 용산아이파크몰점,계절밥상 - 용산아이파크몰점,서울특별시 용산구 한강로3가 40-999 용산역,서울특별시 용산구 한강로3가 40-999 용산역,0
1064,36667369,8942305,계절밥상 동대문롯데피트인점,엔제리너스,서울특별시 중구 을지로6가 21-31 롯데 FiTin,서울특별시 중구 을지로6가 21-31 롯데 FiTin,0
1065,37204578,4034834,계절밥상 잠실점,무스쿠스 잠실점,서울특별시 송파구 신천동 7-18 롯데캐슬골드,서울특별시 송파구 신천동 7-18 롯데캐슬골드,0
1066,38404658,9402393,계절밥상 서울역사점,맥도날드,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사,0


In [209]:
dt.loc[dt['name_duplicate'] == 1]

,place.id,location_id,place.name,name,land.addr_x,land.addr_y,name_duplicate
2,11605455,2241571,가미분식,가미 분식,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,1
22,10976694,5926853,유래회관,유래회관,서울특별시 성동구 홍익동 357 유래회관,서울특별시 성동구 홍익동 357 유래회관,1
24,11398664,4032150,대성갈비,대성갈비,서울특별시 성동구 성수동1가 668-21,서울특별시 성동구 성수동1가 668-21,1
25,11663081,3922609,철길왕갈비살,철길 왕 갈비살,서울특별시 마포구 창전동 4-2,서울특별시 마포구 창전동 4-2,1
26,11677544,4033804,황소고집,황소고집,서울특별시 종로구 관철동 11-11,서울특별시 종로구 관철동 11-11,1
...,...,...,...,...,...,...,...
1029,12447840,4070735,김영모과자점 도곡타워점,김영모과자점 도곡타워점,서울특별시 강남구 도곡동 467-19 현대비전21,서울특별시 강남구 도곡2동 467-19 현대비전21,1
1030,13575110,6818439,미즈컨테이너 강남 웨스트,미즈컨테이너,서울특별시 서초구 서초동 1316-29 타임빌딩,서울특별시 서초구 서초동 1316-29 타임빌딩,1
1032,32817375,4784018,생어거스틴 경복궁점,생어거스틴 경복궁점,서울특별시 종로구 중학동 19 더케이트윈타워,서울특별시 종로구 중학동 19 더케이트윈타워,1
1039,33758018,8948116,아웃닭 홍대점,아웃닭,서울특별시 마포구 서교동 409-17,서울특별시 마포구 서교동 409-17,1


In [207]:
dt.loc[dt['name_duplicate'] == 0].to_excel('./place_check.xlsx')

In [210]:
dt2 = pd.read_excel('./place_check.xlsx')

In [215]:
dt.loc[dt['name_duplicate'] == 0, 'name_duplicate'] = dt2['매칭여부'].values

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [230]:
# 네이버와 트립어드바이저 같은 place
same_place = dt.loc[dt['name_duplicate'] == 1, ['place.id', 'location_id']]

In [233]:
same_place.to_excel(os.path.join("..", "..", "data", "JS_0602_same_place_index.xlsx"))

# ACM 매칭

In [63]:
place = pd.read_csv(os.path.join("..","..", "data","input_data.csv"),  index_col = 0, dtype = {'place.id' : str})

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [82]:
hotel = place.loc[place['category'] != 'EAT'].drop_duplicates('place.id')[['place.id', 'place.name','category', 'land.addr']]

In [93]:
hotel['place.id'] = hotel['place.id'].apply(lambda x : str(int(float(x))))

In [106]:
ta_hotel_info = pd.read_excel(os.path.join('..', '..', 'data', 'TA_호텔_한글이름_주소_정보.xlsx'))

In [107]:
ta_hotel_info['location_id'] = ta_hotel_info['location_id'].astype(str)

In [103]:
ta_hotel = hotel.loc[hotel['land.addr'].isnull()]
nv_hotel = hotel.loc[hotel['land.addr'].notnull()]

In [104]:
ta_hotel.shape, nv_hotel.shape

((592, 4), (221, 4))

In [111]:
ta_hotel_info.rename(columns = {'location_id' : 'place.id', 'name' : 'place.name'}, inplace = True)

In [115]:
ta_hotel = ta_hotel.merge(ta_hotel_info[['place.id', 'name', 'address']], on = 'place.id')

In [119]:
ta_hotel.drop(['place.name', 'land.addr'], axis = 1, inplace = True)

In [123]:
nv_hotel.rename(columns = {'land.addr' : 'address'}, inplace = True)

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [139]:
# 네이버 호텔
addr_list = []
for addr in tqdm(nv_hotel.address.values):
    value = JS.NVgeocoding(addr)
    addr_list.append(value)

100%|██████████| 221/221 [00:17<00:00, 12.44it/s]


In [140]:
nv_hotel

,place.id,place.name,category,address
415515,36728037,31 페이지,ACM,서울특별시 동대문구 전농동 597-23
415516,34300204,Ever8레지던스,ACM,서울특별시 서대문구 대현동 104-5
415517,37490209,E호텔,ACM,서울특별시 서초구 반포대로18길 40 E 호텔
415518,966279439,G2 호텔,ACM,서울특별시 중구 저동2가 48-27
415519,1245880236,Hotel in 9,ACM,서울특별시 강남구 영동대로 618 Hotel in 9
...,...,...,...,...
416757,11659048,퍼시픽호텔,ACM,서울특별시 중구 남산동2가 31-1
416758,1670545829,포포인츠 바이 쉐라톤 서울 강남,ACM,서울특별시 강남구 신사동 587-21
416773,36628778,포포인츠 바이 쉐라톤 서울남산,ACM,서울특별시 용산구 동자동 37-85
416784,11659052,프레지던트 호텔,ACM,서울특별시 중구 을지로1가 188-3


In [143]:
nv_addr = pd.DataFrame(addr_list, columns = ['land.addr', 'road.addr', 'lat', 'lng'])

In [149]:
nv_hotel = nv_hotel.reset_index(drop = True)

In [151]:
nv_hotel = pd.concat([nv_hotel, nv_addr], axis = 1)

In [152]:
# 트립어드바이저 호텔
addr_list = []
for addr in tqdm(ta_hotel.address.values):
    value = JS.NVgeocoding(addr)
    addr_list.append(value)

100%|██████████| 592/592 [00:49<00:00, 11.93it/s]


In [154]:
ta_addr = pd.DataFrame(addr_list, columns = ['land.addr', 'road.addr', 'lat', 'lng'])

In [156]:
ta_hotel = pd.concat([ta_hotel, ta_addr], axis = 1)

In [197]:
total2 = nv_hotel.merge(ta_hotel, on = 'land.addr')[['place.id_x', 'place.name_x', 'place.name_y', 'place.id_y', 'land.addr']]

In [198]:
total2.to_excel('./hotel.distance.check.xlsx')

In [498]:
totel2 = pd.read_excel('./hotel.distance.check.xlsx')

In [499]:
total2

,place.id_x,place.id_y
8,958174664,12707683
9,958174664,12707649
10,958174664,12698628
11,1475741466,12690380
12,1475741466,12707683
14,1475741466,12698628
15,1137153331,12690380
16,1137153331,12707683
17,1137153331,12707649
18,1137153331,12698628


In [202]:
res = joblib.load(os.path.join('..', '..', 'data', 'JS_05_reviews.pkl'))

In [213]:
index = input().split(' ')

 0 0 0 0 0 0 0 0      0             0 0    0  0 0 0 0 0 0   0  0 0  0  0 0 0 0  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0  0 0           0  0  0 0 0 0 0  0  0 0 0 0  0 0 0 0 0 0 0 0 0 0 0 0 0 0    0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [ ]:
total2['is_valid'] = index

In [218]:
total2[total2.is_valid == '0']

,place.id_x,place.name_x,place.name_y,place.id_y,land.addr,is_valid
0,34300204,Ever8레지던스,Ever8 서비스 레지던스,6402635,서울특별시 서대문구 대현동 104-5 신촌푸르지오시티,0
1,966279439,G2 호텔,G2 호텔,12287729,서울특별시 중구 저동2가 48-27,0
2,1245880236,Hotel in 9,호텔인나인,16741053,서울특별시 강남구 삼성동 91-28,0
3,12873138,MEASeoul,여의도파크센터 서울 - 메리어트 이그제큐티브 아파트먼트,659341,서울특별시 영등포구 여의도동 28-3 여의도파크센터,0
4,13286706,강남아르누보시티,강남 아르누보 씨티 II,1643621,서울특별시 서초구 서초동 1330-4 아르누보씨티2,0
...,...,...,...,...,...,...
140,13217405,코트야드 메리어트 서울 타임스퀘어,코트야드 메리어트 서울 타임스퀘어,1557869,서울특별시 영등포구 영등포동4가 442 타임스퀘어,0
141,11659048,퍼시픽호텔,퍼시픽호텔,455976,서울특별시 중구 남산동2가 31-1 퍼시픽호텔,0
142,1670545829,포포인츠 바이 쉐라톤 서울 강남,포포인츠 바이 쉐라톤 서울 강남,13781380,서울특별시 강남구 신사동 587-21 포포인츠바이쉐라톤서울강남,0
143,11659052,프레지던트 호텔,호텔 프레지던트,306136,서울특별시 중구 을지로1가 188-3 프레지던트호텔,0


In [222]:
res.loc[res['place.id'] == '1866553915', 'place.id'] = '34907069'

In [224]:
res.loc[res['place.id'] == '1741224830', 'place.id'] = '13069031'
res.loc[res['place.id'] == '13401813', 'place.id'] = '1287253850'

In [233]:
change_index = total2.loc[total2.is_valid == '0', ['place.id_x', 'place.id_y']]

In [354]:
ta = pd.read_json(os.path.join('..', '..', 'data','TA_User_Review_Seoul_all_v2.json'))

In [239]:
change_index

,place.id_x,place.id_y
0,34300204,6402635
1,966279439,12287729
2,1245880236,16741053
3,12873138,659341
4,13286706,1643621
...,...,...
140,13217405,1557869
141,11659048,455976
142,1670545829,13781380
143,11659052,306136


In [259]:
total2 = total2.loc[total2['is_valid'] == '', ['place.id_x', 'place.id_y']]

In [241]:
ta_review['location.locationId'] = ta_review['location.locationId'].astype(str)

In [243]:
for nv_idx, ta_idx in change_index.values:
    ta_review.loc[ta_review['location.locationId'] == ta_idx, 'location.locationId'] = nv_idx

In [260]:
total2

,place.id_x,place.id_y
8,958174664,12707683
9,958174664,12707649
10,958174664,12698628
11,1475741466,12690380
12,1475741466,12707683
14,1475741466,12698628
15,1137153331,12690380
16,1137153331,12707683
17,1137153331,12707649
18,1137153331,12698628


In [264]:
temp1 = res.merge(total2[['place.id_x']], left_on = 'place.id', right_on = 'place.id_x')

In [265]:
temp2 = res.merge(change_index[['place.id_x']], left_on = 'place.id', right_on = 'place.id_x')

In [267]:
temp3 = ta_review.merge(total2[['place.id_y']], left_on = 'location.locationId', right_on = 'place.id_y')

In [268]:
temp4 = ta_review.merge(change_index[['place.id_x']], left_on = 'location.locationId', right_on = 'place.id_x')

In [270]:
temp1.shape, temp2.shape, temp3.shape, temp4.shape

((316, 25), (966, 25), (5626, 33), (14689, 33))

In [286]:
nv_review = pd.concat([temp1, temp2], axis = 0)

In [288]:
nv_review['photonum'] = nv_review['media'].apply(lambda x: x.count('IMAGE'))

In [290]:
nv_review['createdDate'] = nv_review['visitedDate.date'].astype(str).astype('datetime64') 

In [469]:
ta_review = pd.concat([temp3, temp4], axis = 0)

In [470]:
ta_review['photonum'] = ta_review['photos'].apply(lambda x: len(x))

In [471]:
ta_review['createdDate'] = ta_review['createdDate'].astype(str).astype('datetime64')

In [304]:
nv_review = nv_review[['id', 'rating', 'body', 'idno', 'place.id', 'place.name', 'photonum', 'createdDate']]

In [472]:
ta_review = ta_review[['id', 'rating', 'text', 'userId', 'location.locationId', 'location.name', 'photonum', 'createdDate', 'language']]

In [473]:
nv_review['is_fch'] = 0
ta_review['is_fch'] = 0

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [318]:
nv_review['language'] = 'ko'

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [330]:
for nv_idx, ta_idx in change_index.values:
    ta_hotel.loc[ta_hotel['place.id'] == ta_idx, 'place.id'] = 0

In [335]:
ta_hotel = ta_hotel.loc[ta_hotel['place.id'] != 0]

In [340]:
hotel = pd.concat([nv_hotel, ta_hotel], axis = 0)

In [343]:
final_review = pd.read_excel(os.path.join('..', '..', 'data','JS_10_data_review.xlsx'))

In [344]:
final_review

,id,rating,body,idno,place.id,place.name,photonum,createdDate
0,5ec7ec658160f100109a8368,4.5,NaN,4TXey,1150710065,스타벅스 보라매공원R점,0,2020-05-21 20:42:00
1,5d14eaa963951f37e4243fa5,5.0,none,awWC,1150710065,스타벅스 보라매공원R점,0,2019-06-28 01:11:03
2,5e69024bf896230011648a61,5.0,NaN,1Unzg,1150710065,스타벅스 보라매공원R점,0,2020-03-12 00:22:00
3,5e4f98b95543c700113e18e7,5.0,NaN,LRVl,1150710065,스타벅스 보라매공원R점,0,2020-02-16 17:06:00
4,5c8528bdf9e6b96d632ae302,5.0,2019.03.06\nHH,ESN1,1150710065,스타벅스 보라매공원R점,0,2019-03-11 00:08:48
...,...,...,...,...,...,...,...,...
415510,377783462,5.0,Excellent breakfast something for everyone. Fr...,0D7BA322E49206AF13A9369774B88AE0,10161356,MoMo Cafe,0,2016-05-29 00:00:00
415511,654067981,3.0,자주가는 매장인데 오래된 곳인만큼 아직도 USB충전이 안되는 테이블을 사용합니다. ...,A3EFF472632FABBF7B76AFDBC7E0D27C,3894901,Starbucks Gyeongbokgung Station 3 Line,0,2019-02-22 00:00:00
415512,381624991,5.0,This is an adorable coffee shop in Myeongdong....,11424075023A02372320A7F4A93A55A2,948337,Coffee Love,0,2016-06-11 00:00:00
415513,671052612,4.0,다양한 해산물과 소고기와 함께 뷔페까지 먹을 수 있는 단체 모임장소로도 좋은 샤브샤...,7D786BDC124247789DBC1E9860C54B81,9035619,Haejeokseon Seafood Buffet,4,2019-05-03 00:00:00


In [363]:
ta['id'] = ta['id'].astype(str)

In [366]:
ta[['id', 'language']].drop_duplicates()

,id,language
0,749418557,ja
1,748454819,en
2,745371386,ja
3,744426202,ja
4,743141310,en
...,...,...
144429,418986337,en
144430,418987119,en
144431,418402883,en
144432,418406637,en


In [376]:
final_review['id'] = final_review['id'].astype(str)

In [378]:
final_review = final_review.merge(ta[['id', 'language']].drop_duplicates(), on = 'id', how = 'left')

In [383]:
final_review.loc[final_review['language'].isnull(), 'language'] = 'ko'

In [385]:
place_info = pd.read_excel(os.path.join('..', '..', 'data', 'JS_10_data_places_v2.xlsx'))

In [397]:
place_info.loc[place_info['place.name'].str.contains('점')]

,place.id,place.name,place.category,rating,review.count,is_fch,lat,lng,land.addr
10,34457400,육회자매집 본점1호점,"육류,고기요리-일반",4.583333,357,0,37.570579,126.999887,서울특별시 종로구 종로4가 177 제우빌딩
11,21215392,바토스 이태원점,"멕시코,남미음식-일반",4.422330,940,0,37.534589,126.990177,서울특별시 용산구 이태원동 181-8 이태원교회
12,36932403,타코벨 여의도메리츠점,"멕시코,남미음식-일반",4.488889,48,0,37.524171,126.926440,서울특별시 영등포구 여의도동 25-1 메리츠화재여의도사옥
13,11619957,동화반점,중식당-일반,3.763158,56,0,37.567750,127.008197,서울특별시 중구 을지로6가 18-95
15,11728195,오구반점,중식당-일반,3.826923,33,0,37.566548,126.989878,서울특별시 중구 을지로3가 5-9
...,...,...,...,...,...,...,...,...,...
15161,4030135,후터스 강남점,NaN,3.500000,40,NaN,37.492578,127.029652,서울특별시 서초구 서초동 1337-32 서초동 광일빌딩
15165,4075489,애슐리 천호점,NaN,4.500000,20,NaN,37.541275,127.125912,서울특별시 강동구 천호동 563 동아코아아파트
15166,4049731,캘리포니아피자키친 강남점,NaN,4.000000,20,NaN,37.503483,127.024458,서울특별시 서초구 서초동 1303-31 리젠메디컬타워
15172,3894901,스타벅스 경복궁역점,NaN,4.000000,1,NaN,37.577200,126.972778,서울특별시 종로구 통의동 35-41 우진빌딩


In [410]:
temp = place_info.loc[place_info['is_fch'].isnull()]

In [414]:
temp.loc[(temp['place.name'].apply(lambda x : x[-1] == '점')) & (temp['place.name'].apply(lambda x : len(x.split(' ')) > 1)), 'is_fch'] = 1

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [419]:
temp.loc[temp['is_fch'].isnull(), 'is_fch'] = 0

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [427]:
place_info.loc[temp.index]['is_fch'] = temp['is_fch']

In [435]:
place_info['category'] = 'EAT'

In [436]:
place_info

,place.id,place.name,place.category,rating,review.count,is_fch,lat,lng,land.addr,category
0,11605455,가미분식,종합분식,4.217391,32,0,37.558661,126.945960,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,EAT
1,10976694,유래회관,"육류,고기요리-일반",4.300000,27,0,37.566438,127.031591,서울특별시 성동구 홍익동 357 유래회관,EAT
2,11398664,대성갈비,"육류,고기요리-일반",4.216667,76,0,37.547073,127.044028,서울특별시 성동구 성수동1가 668-21,EAT
3,11663081,철길왕갈비살,"육류,고기요리-일반",4.400000,54,0,37.554546,126.931666,서울특별시 마포구 창전동 4-2,EAT
4,11677544,황소고집,"육류,고기요리-일반",4.379310,69,0,37.568593,126.986046,서울특별시 종로구 관철동 11-11,EAT
...,...,...,...,...,...,...,...,...,...,...
15171,10161356,모모카페 코트야드 서울 남대문,NaN,4.500000,120,NaN,37.561200,126.976882,서울특별시 중구 남대문로4가 17-23 남대문호텔,EAT
15172,3894901,스타벅스 경복궁역점,NaN,4.000000,1,NaN,37.577200,126.972778,서울특별시 종로구 통의동 35-41 우진빌딩,EAT
15173,948337,커피愛,NaN,4.500000,1,NaN,37.562855,126.985572,서울특별시 중구 명동2가 27-1,EAT
15174,9035619,해적선해물부페,NaN,4.000000,1,NaN,37.497622,126.864067,서울특별시 구로구 고척동 76-160 골든타워빌딩,EAT


In [434]:
hotel

,place.id,place.name,category,address,land.addr,road.addr,lat,lng
0,36728037,31 페이지,ACM,서울특별시 동대문구 전농동 597-23,서울특별시 동대문구 전농동 597-23,서울특별시 동대문구 왕산로 238-1,37.582474,127.048659
1,34300204,Ever8레지던스,ACM,서울특별시 서대문구 대현동 104-5,서울특별시 서대문구 대현동 104-5 신촌푸르지오시티,서울특별시 서대문구 신촌역로 7 신촌푸르지오시티,37.557426,126.942566
2,37490209,E호텔,ACM,서울특별시 서초구 반포대로18길 40 E 호텔,서울특별시 서초구 서초동 1593-1 E 호텔,서울특별시 서초구 반포대로18길 40 E 호텔,37.486922,127.012686
3,966279439,G2 호텔,ACM,서울특별시 중구 저동2가 48-27,서울특별시 중구 저동2가 48-27,서울특별시 중구 수표로 24,37.563370,126.990472
4,1245880236,Hotel in 9,ACM,서울특별시 강남구 영동대로 618 Hotel in 9,서울특별시 강남구 삼성동 91-28,서울특별시 강남구 영동대로 618,37.515577,127.060038
...,...,...,...,...,...,...,...,...
585,1150903,영빈 호텔,ACM,서울 중구 퇴계로56길 18,서울특별시 중구 쌍림동 240 영빈관광호텔,서울특별시 중구 퇴계로56길 18 영빈관광호텔,37.563004,127.005544
587,2303934,자자 백팩커스 호스텔,ACM,서울 중구 퇴계로20길 35,서울특별시 중구 남산동2가 18-6,서울특별시 중구 퇴계로20길 35,37.558819,126.986443
589,6998634,이비스 버젯 앰배서더 서울 동대문,ACM,서울 중구 퇴계로 334,서울특별시 중구 광희동2가 238-1,서울특별시 중구 퇴계로 334,37.564187,127.008694
590,299533,이비스 스타일 앰배서더 서울 강남,ACM,서울 강남구 삼성로 431,서울특별시 강남구 대치동 893-1 이비스 스타일 앰배서더 서울 강남,서울특별시 강남구 삼성로 431 이비스 스타일 앰배서더 서울 강남,37.505102,127.057088


In [455]:
place_info.loc[temp.index, 'is_fch'] = temp['is_fch'].values

In [456]:
place_info

,place.id,place.name,place.category,rating,review.count,is_fch,lat,lng,land.addr,category
0,11605455,가미분식,종합분식,4.217391,32,0,37.558661,126.945960,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,EAT
1,10976694,유래회관,"육류,고기요리-일반",4.300000,27,0,37.566438,127.031591,서울특별시 성동구 홍익동 357 유래회관,EAT
2,11398664,대성갈비,"육류,고기요리-일반",4.216667,76,0,37.547073,127.044028,서울특별시 성동구 성수동1가 668-21,EAT
3,11663081,철길왕갈비살,"육류,고기요리-일반",4.400000,54,0,37.554546,126.931666,서울특별시 마포구 창전동 4-2,EAT
4,11677544,황소고집,"육류,고기요리-일반",4.379310,69,0,37.568593,126.986046,서울특별시 종로구 관철동 11-11,EAT
...,...,...,...,...,...,...,...,...,...,...
15171,10161356,모모카페 코트야드 서울 남대문,NaN,4.500000,120,0,37.561200,126.976882,서울특별시 중구 남대문로4가 17-23 남대문호텔,EAT
15172,3894901,스타벅스 경복궁역점,NaN,4.000000,1,1,37.577200,126.972778,서울특별시 종로구 통의동 35-41 우진빌딩,EAT
15173,948337,커피愛,NaN,4.500000,1,0,37.562855,126.985572,서울특별시 중구 명동2가 27-1,EAT
15174,9035619,해적선해물부페,NaN,4.000000,1,0,37.497622,126.864067,서울특별시 구로구 고척동 76-160 골든타워빌딩,EAT


In [458]:
final_review = final_review.merge(place_info[['place.id', 'is_fch']], on = 'place.id', how = 'left')

In [459]:
final_review

,id,rating,body,idno,place.id,place.name,photonum,createdDate,language,is_fch
0,5ec7ec658160f100109a8368,4.5,NaN,4TXey,1150710065,스타벅스 보라매공원R점,0,2020-05-21 20:42:00,ko,1
1,5d14eaa963951f37e4243fa5,5.0,none,awWC,1150710065,스타벅스 보라매공원R점,0,2019-06-28 01:11:03,ko,1
2,5e69024bf896230011648a61,5.0,NaN,1Unzg,1150710065,스타벅스 보라매공원R점,0,2020-03-12 00:22:00,ko,1
3,5e4f98b95543c700113e18e7,5.0,NaN,LRVl,1150710065,스타벅스 보라매공원R점,0,2020-02-16 17:06:00,ko,1
4,5c8528bdf9e6b96d632ae302,5.0,2019.03.06\nHH,ESN1,1150710065,스타벅스 보라매공원R점,0,2019-03-11 00:08:48,ko,1
...,...,...,...,...,...,...,...,...,...,...
415510,377783462,5.0,Excellent breakfast something for everyone. Fr...,0D7BA322E49206AF13A9369774B88AE0,10161356,MoMo Cafe,0,2016-05-29 00:00:00,en,0
415511,654067981,3.0,자주가는 매장인데 오래된 곳인만큼 아직도 USB충전이 안되는 테이블을 사용합니다. ...,A3EFF472632FABBF7B76AFDBC7E0D27C,3894901,Starbucks Gyeongbokgung Station 3 Line,0,2019-02-22 00:00:00,ko,1
415512,381624991,5.0,This is an adorable coffee shop in Myeongdong....,11424075023A02372320A7F4A93A55A2,948337,Coffee Love,0,2016-06-11 00:00:00,en,0
415513,671052612,4.0,다양한 해산물과 소고기와 함께 뷔페까지 먹을 수 있는 단체 모임장소로도 좋은 샤브샤...,7D786BDC124247789DBC1E9860C54B81,9035619,Haejeokseon Seafood Buffet,4,2019-05-03 00:00:00,ko,0


In [477]:
ta_review.columns = ['id', 'rating', 'body', 'idno', 'place.id', 'place.name', 'photonum', 'createdDate', 'language', 'is_fch']

In [513]:
nv_review2 = res.merge(nv_hotel[['place.id']], on = 'place.id')

In [515]:
nv_review2['photonum'] = nv_review2['media'].apply(lambda x: x.count('IMAGE'))
nv_review2['createdDate'] = nv_review2['visitedDate.date'].astype(str).astype('datetime64') 
nv_review2['is_fch'] = 0
nv_review2 = nv_review2[['id', 'rating', 'body', 'idno', 'place.id', 'place.name', 'photonum', 'createdDate']]

In [523]:
place_info['place.id'] = place_info['place.id'].astype(str)

In [536]:
nv_review2['language'] = 'ko'

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [540]:
nv_review = pd.concat([nv_review, nv_review2], axis = 0)

In [554]:
ta_review2 = ta.merge(ta_hotel[['place.id']], left_on = 'location.locationId', right_on = 'place.id')

In [556]:
ta_review2['photonum'] = ta_review2['photos'].apply(lambda x: len(x))
ta_review2['createdDate'] = ta_review2['createdDate'].astype(str).astype('datetime64')
ta_review2 = ta_review2[['id', 'rating', 'text', 'userId', 'location.locationId', 'location.name', 'photonum', 'createdDate', 'language']]
ta_review2['is_fch'] = 0

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [561]:
ta_review2.columns = ['id', 'rating', 'body', 'idno', 'place.id', 'place.name', 'photonum', 'createdDate', 'language', 'is_fch']

In [563]:
ta_review = pd.concat([ta_review, ta_review2], axis = 0)

In [565]:
reviews = pd.concat([final_review, nv_review, ta_review] , axis = 0)

In [572]:
reviews['is_local'] = reviews['language'].apply(lambda x : 1 if x == 'ko' else 0)

In [568]:
place_info

,place.id,place.name,place.category,rating,review.count,is_fch,lat,lng,land.addr,category
0,11605455,가미분식,종합분식,4.217391,32,0,37.558661,126.945960,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,EAT
1,10976694,유래회관,"육류,고기요리-일반",4.300000,27,0,37.566438,127.031591,서울특별시 성동구 홍익동 357 유래회관,EAT
2,11398664,대성갈비,"육류,고기요리-일반",4.216667,76,0,37.547073,127.044028,서울특별시 성동구 성수동1가 668-21,EAT
3,11663081,철길왕갈비살,"육류,고기요리-일반",4.400000,54,0,37.554546,126.931666,서울특별시 마포구 창전동 4-2,EAT
4,11677544,황소고집,"육류,고기요리-일반",4.379310,69,0,37.568593,126.986046,서울특별시 종로구 관철동 11-11,EAT
...,...,...,...,...,...,...,...,...,...,...
15171,10161356,모모카페 코트야드 서울 남대문,NaN,4.500000,120,0,37.561200,126.976882,서울특별시 중구 남대문로4가 17-23 남대문호텔,EAT
15172,3894901,스타벅스 경복궁역점,NaN,4.000000,1,1,37.577200,126.972778,서울특별시 종로구 통의동 35-41 우진빌딩,EAT
15173,948337,커피愛,NaN,4.500000,1,0,37.562855,126.985572,서울특별시 중구 명동2가 27-1,EAT
15174,9035619,해적선해물부페,NaN,4.000000,1,0,37.497622,126.864067,서울특별시 구로구 고척동 76-160 골든타워빌딩,EAT


In [576]:
nv_hotel = nv_hotel[['place.id', 'place.name', 'lat', 'lng', 'land.addr']]

In [577]:
ta_hotel = ta_hotel[['place.id', 'place.name', 'lat', 'lng', 'land.addr']]

In [579]:
hotel = pd.concat([nv_hotel, ta_hotel], axis = 0)

In [580]:
hotel['category'] = 'ACM'

In [583]:
place_info = pd.concat([place_info, hotel], axis = 0)

In [585]:
place_info

,place.id,place.name,place.category,rating,review.count,is_fch,lat,lng,land.addr,category
0,11605455,가미분식,종합분식,4.217391,32.0,0,37.558661,126.945960,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,EAT
1,10976694,유래회관,"육류,고기요리-일반",4.300000,27.0,0,37.566438,127.031591,서울특별시 성동구 홍익동 357 유래회관,EAT
2,11398664,대성갈비,"육류,고기요리-일반",4.216667,76.0,0,37.547073,127.044028,서울특별시 성동구 성수동1가 668-21,EAT
3,11663081,철길왕갈비살,"육류,고기요리-일반",4.400000,54.0,0,37.554546,126.931666,서울특별시 마포구 창전동 4-2,EAT
4,11677544,황소고집,"육류,고기요리-일반",4.379310,69.0,0,37.568593,126.986046,서울특별시 종로구 관철동 11-11,EAT
...,...,...,...,...,...,...,...,...,...,...
585,1150903,영빈 호텔,NaN,NaN,NaN,NaN,37.563004,127.005544,서울특별시 중구 쌍림동 240 영빈관광호텔,ACM
587,2303934,자자 백팩커스 호스텔,NaN,NaN,NaN,NaN,37.558819,126.986443,서울특별시 중구 남산동2가 18-6,ACM
589,6998634,이비스 버젯 앰배서더 서울 동대문,NaN,NaN,NaN,NaN,37.564187,127.008694,서울특별시 중구 광희동2가 238-1,ACM
590,299533,이비스 스타일 앰배서더 서울 강남,NaN,NaN,NaN,NaN,37.505102,127.057088,서울특별시 강남구 대치동 893-1 이비스 스타일 앰배서더 서울 강남,ACM


In [589]:
place_info.loc[place_info['place.id'] == '1150710065']

,place.id,place.name,place.category,rating,review.count,is_fch,lat,lng,land.addr,category
8393,1150710065,스타벅스 보라매공원R점,스타벅스,4.590717,158.0,1,37.492103,126.923551,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",EAT


In [595]:
reviews['place.id'] = reviews['place.id'].astype(str)

In [594]:
place_info['place.id'] = place_info['place.id'].astype(str)

In [597]:
reviews = reviews.merge(place_info[['place.id', 'place.name']], on = 'place.id', how = 'left')

In [599]:
reviews = reviews.drop('place.name_x', axis = 1).rename(columns = {'place.name_y' : 'place.name'})

In [602]:
reviews.to_excel(os.path.join('..', '..', 'data', 'JS_11_data_review.xlsx'))

In [603]:
place_info.to_excel(os.path.join('..', '..', 'data', 'JS_11_data_place.xlsx'))

In [604]:
reviews.head()

,id,rating,body,idno,place.id,photonum,createdDate,language,is_fch,is_local,place.name
0,5ec7ec658160f100109a8368,4.5,NaN,4TXey,1150710065,0,2020-05-21 20:42:00,ko,1,1,스타벅스 보라매공원R점
1,5d14eaa963951f37e4243fa5,5.0,none,awWC,1150710065,0,2019-06-28 01:11:03,ko,1,1,스타벅스 보라매공원R점
2,5e69024bf896230011648a61,5.0,NaN,1Unzg,1150710065,0,2020-03-12 00:22:00,ko,1,1,스타벅스 보라매공원R점
3,5e4f98b95543c700113e18e7,5.0,NaN,LRVl,1150710065,0,2020-02-16 17:06:00,ko,1,1,스타벅스 보라매공원R점
4,5c8528bdf9e6b96d632ae302,5.0,2019.03.06\nHH,ESN1,1150710065,0,2019-03-11 00:08:48,ko,1,1,스타벅스 보라매공원R점


In [606]:
place_info

,place.id,place.name,place.category,rating,review.count,is_fch,lat,lng,land.addr,category
0,11605455,가미분식,종합분식,4.217391,32.0,0,37.558661,126.945960,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,EAT
1,10976694,유래회관,"육류,고기요리-일반",4.300000,27.0,0,37.566438,127.031591,서울특별시 성동구 홍익동 357 유래회관,EAT
2,11398664,대성갈비,"육류,고기요리-일반",4.216667,76.0,0,37.547073,127.044028,서울특별시 성동구 성수동1가 668-21,EAT
3,11663081,철길왕갈비살,"육류,고기요리-일반",4.400000,54.0,0,37.554546,126.931666,서울특별시 마포구 창전동 4-2,EAT
4,11677544,황소고집,"육류,고기요리-일반",4.379310,69.0,0,37.568593,126.986046,서울특별시 종로구 관철동 11-11,EAT
...,...,...,...,...,...,...,...,...,...,...
585,1150903,영빈 호텔,NaN,NaN,NaN,NaN,37.563004,127.005544,서울특별시 중구 쌍림동 240 영빈관광호텔,ACM
587,2303934,자자 백팩커스 호스텔,NaN,NaN,NaN,NaN,37.558819,126.986443,서울특별시 중구 남산동2가 18-6,ACM
589,6998634,이비스 버젯 앰배서더 서울 동대문,NaN,NaN,NaN,NaN,37.564187,127.008694,서울특별시 중구 광희동2가 238-1,ACM
590,299533,이비스 스타일 앰배서더 서울 강남,NaN,NaN,NaN,NaN,37.505102,127.057088,서울특별시 강남구 대치동 893-1 이비스 스타일 앰배서더 서울 강남,ACM


In [638]:
temp = reviews.groupby('place.id').agg({
    'id' : [('count', 'count')],
    'rating' : [('rating', np.mean)]
}).reset_index()

In [639]:
temp.columns = ['place.id', 'review.count', 'rating']

In [643]:
place_info = place_info.merge(temp, on = 'place.id', how = 'left')

In [647]:
place_info = place_info.drop(['review.count_x', 'rating_x'], axis = 1).rename(columns = {'review.count_y' : 'review.count', 'rating_y' : 'rating'})

In [652]:
place_info.loc[place_info['is_fch'].isnull(), 'is_fch'] = 0

In [654]:
place_info.to_excel(os.path.join('..', '..', 'data', 'JS_11_data_place.xlsx'))

In [655]:
reviews

,id,rating,body,idno,place.id,photonum,createdDate,language,is_fch,is_local,place.name
0,5ec7ec658160f100109a8368,4.5,NaN,4TXey,1150710065,0,2020-05-21 20:42:00,ko,1,1,스타벅스 보라매공원R점
1,5d14eaa963951f37e4243fa5,5.0,none,awWC,1150710065,0,2019-06-28 01:11:03,ko,1,1,스타벅스 보라매공원R점
2,5e69024bf896230011648a61,5.0,NaN,1Unzg,1150710065,0,2020-03-12 00:22:00,ko,1,1,스타벅스 보라매공원R점
3,5e4f98b95543c700113e18e7,5.0,NaN,LRVl,1150710065,0,2020-02-16 17:06:00,ko,1,1,스타벅스 보라매공원R점
4,5c8528bdf9e6b96d632ae302,5.0,2019.03.06\nHH,ESN1,1150710065,0,2019-03-11 00:08:48,ko,1,1,스타벅스 보라매공원R점
...,...,...,...,...,...,...,...,...,...,...,...
479686,328361424,3.0,Had a sunny room with window that look out ont...,D16BB514559A76BBC8FFF38409760136,6378634,0,2015-11-22 00:00:00,en,0,0,24게스트하우스 가로수길점
479687,315070099,2.0,Very old tired looking guesthouse run by a wif...,D16BB514559A76BBC8FFF38409760136,2226404,0,2015-09-30 00:00:00,en,0,0,H2O 게스트하우스
479688,312636488,3.0,Bong's house is around 10mins walk from Hyewa ...,21C47F0112F45C4A3BA1EDD32821E0F7,1625995,9,2015-09-22 00:00:00,en,0,0,봉 하우스
479689,170979510,3.0,It was located in a good location and near to ...,1EF4776FF04831410FD949CB4A2FDF5C,6892917,1,2013-08-06 00:00:00,en,0,0,24게스트하우스 신촌


In [656]:
place_info

,place.id,place.name,place.category,is_fch,lat,lng,land.addr,category,review.count,rating
0,11605455,가미분식,종합분식,0,37.558661,126.945960,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,EAT,32.0,4.093750
1,10976694,유래회관,"육류,고기요리-일반",0,37.566438,127.031591,서울특별시 성동구 홍익동 357 유래회관,EAT,27.0,4.185185
2,11398664,대성갈비,"육류,고기요리-일반",0,37.547073,127.044028,서울특별시 성동구 성수동1가 668-21,EAT,76.0,4.217105
3,11663081,철길왕갈비살,"육류,고기요리-일반",0,37.554546,126.931666,서울특별시 마포구 창전동 4-2,EAT,54.0,4.407407
4,11677544,황소고집,"육류,고기요리-일반",0,37.568593,126.986046,서울특별시 종로구 관철동 11-11,EAT,69.0,4.326087
...,...,...,...,...,...,...,...,...,...,...
15885,1150903,영빈 호텔,NaN,0,37.563004,127.005544,서울특별시 중구 쌍림동 240 영빈관광호텔,ACM,2.0,5.000000
15886,2303934,자자 백팩커스 호스텔,NaN,0,37.558819,126.986443,서울특별시 중구 남산동2가 18-6,ACM,4.0,4.250000
15887,6998634,이비스 버젯 앰배서더 서울 동대문,NaN,0,37.564187,127.008694,서울특별시 중구 광희동2가 238-1,ACM,379.0,3.947230
15888,299533,이비스 스타일 앰배서더 서울 강남,NaN,0,37.505102,127.057088,서울특별시 강남구 대치동 893-1 이비스 스타일 앰배서더 서울 강남,ACM,13.0,4.230769
